# I will use MySQL format

In [ ]:
# Q1
# App Click-through Rate (CTR)
SELECT app_id, sum(event_type = 'click') * 100.0 / sum(event_type = 'impression') asa ctr FROM events
where year(timestamp) = 2019
group by app_id;
# Tips: 1) you could also use sum (IF(event_type = 'click',1,0))

In [ ]:
# Q2
select u.city, count(Distinct order_id) as cnt from trades t
inner join users u on u.user_id = t.user_id
where status = 'complete'
group by u.city
order by cnt DESC
limit 3

In [ ]:
# Q3
select sum(device_type = 'tablet' or device_type = 'phone') as mobile, sum(device_type = 'laptop') from viewership
-------------
select sum(if(device_type = 'tablet' or device_type = 'phone',1,0)) as mobile, 
sum(if(device_type = 'laptop',1,0)) from viewership


In [ ]:
# Q4
select product_id, trans_date, sum(spend) over (partition by product_id order by trans_date
                                               RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
from total_trans
order by product_id, trans_date ASC

# The default window frame without the ORDER BY is the whole partition. 
# But when you use the ORDER BY, the default window frame is RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW.

# Thus, "RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW" was unnecessary

In [ ]:
# Q5
select user_id, count(product_id) as cnt, sum(spend) as sm from user_transactions
group by user_id
having sum(spend) > 1000
order by cnt DESC
LIMIT 10

In [ ]:
# Q6
WITH CTE as (
    select user_id, count(tweet_id) as cnt_tw from tweets
    where year(tweet_date) = 2020
    group by user_id)
select count(user_id), cnt_tw from CTE
group by cnt_tw
order by cnt_tw ASC

In [ ]:
# Q7
select count(distinct user_id) from (
        select user_id,
        rank() over (partition by user_id, product_id order by datetime) as cnt from purchases) table
where cnt >= 2

# Tip: rank and dense_rank will assign the same number if the value of order by paarameter is equal. For instnace:
# for equal user_id and product_id if two rows had the same datetime will get the same rank regardless of other columns

In [ ]:
# Q8
select count(distinct cmp) from (select j1.job_id, j2.job_id, j1.company_id as cmp from job_listings j1
inner join job_listings j2 on j1.company_id = j2.company_id
where j1.title = j2.title and j1.description = j2.description and j1.job_id != j2.job_id) t

-------------------
with CTE as (select company_id, row_number() over(partition by company_id, title, description order by post_date) as RN 
from job_listings)

select count(distinct company_id) from CTE 
where RN >=2


# Don't use "table" as alias

In [ ]:
# Q9
with CTE as (
select user_id, spend,
        row_number() over(user_id order by transaction_date ASC) as RN
from user_transactions
)
select user_id from CTE
where RN = 1 and spend >= 50

--------------------------
WITH CTE as (
select user_id, min(transaction_date) as mn from user_transactions
group by user_id)

select C.user_id from CTE C
inner join user_transactions u on u.user_id = C.user_id and u.transaction_date = C.mn
where u.spend >= 50


In [ ]:
# Q10
with CTE as (
select user_id, cast(tweet_date as date) as dt, COUNT(*) as cnt from tweets
    group by user_id, dt
)
select user_id, dt,
        AVG(cnt) over (partition by user_id order by user_id, dt rows between 6 preceding and current row) as moving_average

from CTE

# Tips: 1) you can use several columns in partition by and order by clause of Window function. 
# Just make sure they are ssperated by a comma
# 2) You can use same columns in paartition by and order by statement
    